<a href="https://colab.research.google.com/github/sarikasaxena30/Fake-News-Detection/blob/main/Fakenewsdetection_optuna_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
vers = torch.__version__
print("Torch vers: ", vers)

# PyG installation
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

import torch_geometric

Torch vers:  2.0.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.0 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.datasets import UPFD
import numpy as np
from torch_geometric.loader import DataLoader

In [ ]:
import torch.nn.functional as F
from torch.nn import ReLU, LeakyReLU, Softmax, Linear, SELU, GELU, ELU
from torch_geometric.nn import SAGEConv, global_max_pool, TopKPooling,GCNConv

In [ ]:
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
test_data_gos = UPFD(root=".", name="gossipcop", feature="bert",split="test")
train_data_gos = UPFD(root=".", name="gossipcop", feature="bert", split="train")
val_data_gos = UPFD(root=".", name="gossipcop", feature="bert", split="val")

Extracting ./gossipcop/raw/uc
Processing...
Done!


In [ ]:
train_loader = DataLoader(train_data_gos, batch_size=32, shuffle=True)
test_loader = DataLoader(train_data_gos, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data_gos, batch_size=32, shuffle=True)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.leaky1 = LeakyReLU()
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.leaky2 = LeakyReLU()
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        self.leaky3 = LeakyReLU()

        self.full1 = Linear(hidden_channels[2],hidden_channels[3])
        self.ge1 = GELU()
        self.full2 = Linear(hidden_channels[3],hidden_channels[4])
        self.ge2 = GELU()
        self.full3 = Linear(hidden_channels[4],out_channels)


    def forward(self, x, edge_index, batch):
        h = self.leaky1(self.conv1(x, edge_index))
        h = self.leaky2(self.conv2(h, edge_index))
        h = self.leaky3(self.conv3(h, edge_index))

        h = global_max_pool(h,batch)

        h = self.ge1(self.full1(h))
        h = self.ge2(self.full2(h))
        h = self.full3(h)

        return torch.sigmoid(h)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def objective(trial):

  model = Net(768,[trial.suggest_int(name="layer_size1",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size2",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size3",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size4",low=64,high=256,step=64),
                   trial.suggest_int(name="layer_size5",low=64,high=256,step=64),

  ],1).to(device)
  optimizer = torch.optim.Adam(model.parameters(),lr=trial.suggest_loguniform('learning_rate', 1e-6, 1e-3),betas=(trial.suggest_loguniform('b1', 1-1e-1,1-1e-3),0.99))
  lossff = torch.nn.BCELoss()

  total_loss = 0
  weighted_loss = 0
  exp_param = 0.8

  wloss = []

  for i in range(400):
    print("Epoch:", i)
    model.train()
    for data in train_loader:
          data = data.to(device)
          optimizer.zero_grad()
          out = model(data.x, data.edge_index, data.batch)
          loss = lossff(torch.reshape(out,(-1,)), data.y.float())
          loss.backward()
          optimizer.step()
          total_loss += float(loss) * data.num_graphs
    print("Train: ",total_loss / len(train_loader.dataset))

    model.eval()
    total_loss = 0
    for data in val_loader:
          data = data.to(device)
          out = model(data.x, data.edge_index, data.batch)
          loss = lossff(torch.reshape(out,(-1,)), data.y.float())
          total_loss += float(loss) * data.num_graphs

    print("Test", total_loss / len(val_loader.dataset))

    weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(val_loader.dataset))
    print(weighted_loss/(1-exp_param**(i+1)))
    wloss.append(weighted_loss/(1-exp_param**(i+1)))

    if(i-30>=0 and wloss[i-20]-weighted_loss<0.01):
      break

  return weighted_loss

In [ ]:
import optuna
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(),pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=10)

[I 2023-04-16 06:23:45,712] A new study created in memory with name: no-name-28f112cb-fd10-45a6-a7fc-76f451b7ad87
<ipython-input-10-543a82f29da4>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  optimizer = torch.optim.Adam(model.parameters(),lr=trial.suggest_loguniform('learning_rate', 1e-6, 1e-3),betas=(trial.suggest_loguniform('b1', 1-1e-1,1-1e-3),0.99))


Epoch: 0
Train:  0.6934147110352149
Test 0.6920694894406385
0.6920694894406385
Epoch: 1
Train:  1.038651126644987
Test 0.6915197005638709
0.6917640511757677
Epoch: 2
Train:  1.0375781332180176
Test 0.6908056698875986
0.691371271959305
Epoch: 3
Train:  1.036061392926471
Test 0.6897645472606897
0.6908269885248146
Epoch: 4
Train:  1.0343688453510131
Test 0.6886534044594118
0.6901803964063583
Epoch: 5
Train:  1.0319084457425407
Test 0.6868456466730698
0.6892764938201388
Epoch: 6
Train:  1.0287014187037289
Test 0.6844488377972837
0.6880547428708974
Epoch: 7
Train:  1.0244943370749227
Test 0.6807808718838535
0.6863066948875365
Epoch: 8
Train:  1.018651009479285
Test 0.6764253906277946
0.6840240645821329
Epoch: 9
Train:  1.011189103781522
Test 0.6694483444804237
0.6807582571599593
Epoch: 10
Train:  1.0005602401036482
Test 0.6607464062861907
0.6763797785352154
Epoch: 11
Train:  0.9863411884604792
Test 0.6492130743278252
0.6705455097990508
Epoch: 12
Train:  0.9680837333857358
Test 0.63419630501

[I 2023-04-16 06:24:31,952] Trial 0 finished with value: 0.1359454151202222 and parameters: {'layer_size1': 512, 'layer_size2': 384, 'layer_size3': 256, 'layer_size4': 64, 'layer_size5': 64, 'learning_rate': 8.725039194497957e-06, 'b1': 0.9161235464257066}. Best is trial 0 with value: 0.1359454151202222.


Train:  0.19229142748556294
Test 0.140060434847961
0.13594541752216982
Epoch: 0
Train:  0.6836875290661067
Test 0.5594880904033507
0.5594880904033507
Epoch: 1
Train:  0.7037098809476301
Test 0.23874806948415525
0.38129918989268663
Epoch: 2
Train:  0.3325709589780905
Test 0.15170147124807531
0.28720176421866556
Epoch: 3
Train:  0.2412189018406015
Test 0.1435642197946489
0.2385440594679824
Epoch: 4
Train:  0.22041506393925175
Test 0.14968347809731197
0.2121100454952699
Epoch: 5
Train:  0.20094118520255871
Test 0.19106734428391026
0.20640630351543654
Epoch: 6
Train:  0.26404970005024975
Test 0.22499165857285808
0.21110976106831408
Epoch: 7
Train:  0.2666235258222827
Test 0.1560648519191004
0.19788143428001204
Epoch: 8
Train:  0.20198687840075719
Test 0.1416940406400236
0.18490186760207747
Epoch: 9
Train:  0.1686488956089546
Test 0.15271021485765338
0.17768906866444925
Epoch: 10
Train:  0.1518862579536416
Test 0.1666542769720157
0.17527471929423913
Epoch: 11
Train:  0.14222157013435394
Tes

[I 2023-04-16 06:24:47,405] Trial 1 finished with value: 0.29991363899795165 and parameters: {'layer_size1': 384, 'layer_size2': 512, 'layer_size3': 256, 'layer_size4': 64, 'layer_size5': 64, 'learning_rate': 0.0006065539019270989, 'b1': 0.9483548996636576}. Best is trial 0 with value: 0.1359454151202222.


...

Train:  0.18642123758574544
Test 0.1371981599828699
0.13702473439990154
Epoch: 0
Train:  0.693120056456262
Test 0.6929522224398323
0.6929522224398322
Epoch: 1
Train:  1.0394761505581083
Test 0.6928776727491246
0.6929108059449948
Epoch: 2
Train:  1.039330536301756
Test 0.6928100970201877
0.6928695317954836
Epoch: 3
Train:  1.0391794840494792
Test 0.6927417638974313
0.6928262499872002
Epoch: 4
Train:  1.0390733028506185
Test 0.6926871022898635
0.6927848566931329
Epoch: 5
Train:  1.038926702497643
Test 0.6926230552432301
0.6927409995042226
Epoch: 6
Train:  1.0387956566644676
Test 0.6925544926535079
0.6926937995958569
Epoch: 7
Train:  1.038660283708747
Test 0.6924829133700975
0.6926431196700905
Epoch: 8
Train:  1.0385252368319167
Test 0.6924044629593036
0.6925879887859168
Epoch: 9
Train:  1.0383701011156425
Test 0.6923109716548151
0.6925259208677276
Epoch: 10
Train:  1.0382143128049242
Test 0.6922115292741265
0.6924571337834612
Epoch: 11
Train:  1.038049237850385
Test 0.6921058474006233
0.

[I 2023-04-16 06:27:25,204] Trial 8 finished with value: 0.6890591921100013 and parameters: {'layer_size1': 512, 'layer_size2': 256, 'layer_size3': 256, 'layer_size4': 64, 'layer_size5': 64, 'learning_rate': 1.9360183517613576e-06, 'b1': 0.9903978011454345}. Best is trial 0 with value: 0.1359454151202222.


Train:  1.0332397523817125
Test 0.6889379474269601
0.6897422797779413
Epoch: 0
Train:  0.6931207885235657
Test 0.6916176871065691
0.6916176871065691
Epoch: 1
Train:  1.0369379841364348
Test 0.6901710946044642
0.6908140246053999
Epoch: 2
Train:  1.0337493806094913
Test 0.6867474286984174
0.6891473869386038
Epoch: 3
Train:  1.0264917013846038
Test 0.6791440396955162
0.6857587191733302
Epoch: 4
Train:  1.0110916936353886
Test 0.6612237307614895
0.6784601148147388
Epoch: 5
Train:  0.9761450000298328
Test 0.6192325008657825
0.6624061384429383
Epoch: 6
Train:  0.9003245003057487
Test 0.543749172827263
0.6323772259254066
Epoch: 7
Train:  0.7933690736800323
Test 0.4670913495641925
0.5926559216342129
Epoch: 8
Train:  0.6817188791848801
Test 0.39762306486293947
0.547602364157057
Epoch: 9
Train:  0.5884612797053306
Test 0.330574458362637
0.49897551480646524
Epoch: 10
Train:  0.507801925811248
Test 0.2866052535154444
0.4525101151060872
Epoch: 11
Train:  0.45363031833981854
Test 0.25487925499786823

[I 2023-04-16 06:27:56,440] Trial 9 finished with value: 0.12781817140888083 and parameters: {'layer_size1': 384, 'layer_size2': 512, 'layer_size3': 512, 'layer_size4': 64, 'layer_size5': 128, 'learning_rate': 1.8041786734871513e-05, 'b1': 0.942414121422847}. Best is trial 9 with value: 0.12781817140888083.


Train:  0.15246355670057374
Test 0.1280667048993251
0.12781832811369512


In [ ]:
best_trial = study.best_trial

In [ ]:
print(best_trial.params)

{'layer_size1': 384, 'layer_size2': 512, 'layer_size3': 512, 'layer_size4': 64, 'layer_size5': 128, 'learning_rate': 1.8041786734871513e-05, 'b1': 0.942414121422847}


In [ ]:
import json

In [ ]:
with open('mydata.json', 'w') as f:
    json.dump(best_trial.params, f)

In [ ]:
model = Net(768,[384,256,512,64,192],1).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1.713644852336342e-05,betas=(0.9308401877808863,0.99))
lossff = torch.nn.BCELoss()

total_loss = 0
weighted_loss = 0
exp_param = 0.8

wloss = []

for i in range(800):
  print("Epoch:", i)
  model.train()
  for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
  print("Train: ",total_loss / len(train_loader.dataset))

  model.eval()
  total_loss = 0
  for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        total_loss += float(loss) * data.num_graphs

  print("Test", total_loss / len(val_loader.dataset))

  weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(val_loader.dataset))
  print(weighted_loss/(1-exp_param**(i+1)))
  wloss.append(weighted_loss/(1-exp_param**(i+1)))

  if(i-30>=0 and wloss[i-20]-weighted_loss<0.01):
    break

print(weighted_loss)

Epoch: 0
Train:  0.6925759520722833
Test 0.6925863686935368
0.6925863686935368
Epoch: 1
Train:  1.037886774801946
Test 0.6915017303093013
0.6919837918134061
Epoch: 2
Train:  1.035775717034008
Test 0.6895549251483037
0.6909883546555773
Epoch: 3
Train:  1.031951198101917
Test 0.6858244481540862
0.689239063835289
Epoch: 4
Train:  1.02443078487784
Test 0.6786982467322996
0.6861034090569128
Epoch: 5
Train:  1.0106353102586207
Test 0.6640936345844478
0.6801375364550867
Epoch: 6
Train:  0.9833837880756392
Test 0.6345520486761799
0.6686010656083978
Epoch: 7
Train:  0.9306704917233506
Test 0.5799172134189815
0.647288668227071
Epoch: 8
Train:  0.8403752478924427
Test 0.49931158742188536
0.6131052284429896
Epoch: 9
Train:  0.7190413482132412
Test 0.4095353399004255
0.567493756193643
Epoch: 10
Train:  0.5885355036992294
Test 0.32235305715393237
0.5138583719620381
Epoch: 11
Train:  0.47343233435145227
Test 0.2591274885030893
0.4591528611958181
Epoch: 12
Train:  0.38723551812189405
Test 0.2138525075

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
@torch.no_grad()
def val(model):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(val_loader.dataset), accuracy, f1

@torch.no_grad()
def test(model):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1

In [ ]:
val(model)

(0.04608704474491951, 0.9835164835164835, 0.984182776801406)

In [ ]:
final = DataLoader(train_data_gos+val_data_gos, batch_size=32, shuffle=True)

In [ ]:
model = Net(768,[384,256,512,64,192],1).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1.713644852336342e-05,betas=(0.9308401877808863,0.99))
lossff = torch.nn.BCELoss()

total_loss = 0
weighted_loss = 0
exp_param = 0.8

wloss = []

for i in range(70):
  print("Epoch:", i)
  model.train()
  total_loss = 0
  for data in final:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
  print("Train: ",total_loss / len(final.dataset))

  model.eval()
  total_loss = 0
  for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        total_loss += float(loss) * data.num_graphs

  print("Test", total_loss / len(test_loader.dataset))

  weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(test_loader.dataset))
  print(weighted_loss/(1-exp_param**(i+1)))
  wloss.append(weighted_loss/(1-exp_param**(i+1)))

  if(i-30>=0 and wloss[i-20]-weighted_loss<0.001):
      break

  print(weighted_loss)

Epoch: 0
Train:  0.6928040554817488
Test 0.6921507486930261
0.6921507486930261
0.13843014973860518
Epoch: 1
Train:  0.691360590076563
Test 0.6899105952336237
0.6909062189933581
0.24872623883760886
Epoch: 2
Train:  0.6876316991190042
Test 0.6837955222025023
0.6879919989971057
0.33574009551058753
Epoch: 3
Train:  0.6770928546477004
Test 0.6666947620692271
0.6807774878426753
0.40193102882231546
Epoch: 4
Train:  0.6461465028324989
Test 0.6170685142824501
0.6618255085589339
0.4449585259143424
Epoch: 5
Train:  0.5650898526003073
Test 0.5063463591393971
0.6196820145927571
0.45723609255935327
Epoch: 6
Train:  0.4412529022411258
Test 0.4001550111141834
0.5641255864598678
0.44581987627031927
Epoch: 7
Train:  0.3566672228304021
Test 0.3528291540486472
0.5133470803211593
0.4272217318259849
Epoch: 8
Train:  0.3165626852649181
Test 0.3161587009936462
0.4677955864399095
0.4050091256595172
Epoch: 9
Train:  0.2832153856535971
Test 0.28641652296750975
0.4271561471819071
0.38129060512111573
Epoch: 10
Tra

In [ ]:
test(model)

(0.05736993974750186, 0.9871794871794872, 0.9869402985074628)